In [ ]:
import pandas as pd
import json
import re

In [ ]:
input_filename = 'chewy.json'
output_filename = 'preprocessed_chewy.json'

In [ ]:
with open(input_filename) as json_file:
    data = json.load(json_file)

In [ ]:
df = pd.DataFrame(data['products'])
df

In [ ]:
# Удаляем нулевые столбцы
df.dropna(axis=1, how='all', inplace=True)
print(f'column number = {len(df.columns)}')

In [ ]:
# Удаляем столбцы, в каждом из которых находятся только одинаковые значения
df = df.loc[:, (df != df.iloc[0]).any()]
print(f'column number = {len(df.columns)}')

In [ ]:
# Удаляем столбец 'mpn' (manufacturer part number), который является источником для 'productcode'
df = df.drop(axis=1, labels='mpn')
print(f'column number = {len(df.columns)}')

In [ ]:
# Удаляем строки с пустым описанием
df = df[df['fulldescr'] != '']
print(f"dataset size = {len(df)}")

In [ ]:
# Добавляем столбец 'supplier_categories_string', в котором в виде строки представляем иерархическую категорию
df['supplier_categories_string'] = [' - '.join(map(str, l)) for l in df['supplier_categories']]
print(f'df["supplier_categories"][0] = {df["supplier_categories"][0]}')
print(f'\ndf["supplier_categories_string"][0] = {df["supplier_categories_string"][0]}')

In [ ]:
# Удаляем строки с пустой категорией
df = df[df['supplier_categories'].str.len() != 0]
print(f"dataset size = {len(df)}")

In [ ]:
# Удаляем дубликаты (сохраняя только первый элемент из дублирующихся: keep='first')
df.drop_duplicates(subset=['product', 'fulldescr', 'supplier_categories_string'], keep='first', inplace=True)
print(f"dataset size = {len(df)}")

In [ ]:
# Перенумеровываем индексы, чтобы они начинались с нуля
df = df.reset_index(drop=True)

In [ ]:
df.fulldescr = df.fulldescr.apply(lambda x: re.sub(r'<\S*>', "", x))
df

In [ ]:
df.to_json(output_filename, indent=2)